<a class="anchor" id="0"></a>
# Example of Geostatistical analysis with SciKit-GStat

### Geostatistical analysis with SciKit-GStat from the [Tutorial](https://scikit-gstat.readthedocs.io/en/latest/tutorials/tutorials.html) from 

**Mirko Mälicke, Egil Möller, Helge David Schneider, & Sebastian Müller. (2021, May 28).**

    mmaelicke/scikit-gstat: A scipy flavoured geostatistical variogram analysis toolbox (Version v0.6.0). Zenodo. 

http://doi.org/10.5281/zenodo.4835779

<a class="anchor" id="0.1"></a>

## Table of Contents

1. [About SciKit-GStat and Geostatistical analysis](#1)
1. [Import libraries](#2)
1. [Creation dataset](#3)
1. [Variogram building](#4)
1. [Visualization of the result of geostatistical data interpolation](#5)

## 1. About SciKit-GStat and Geostatistical analysis <a class="anchor" id="1"></a>

[Back to Table of Contents](#0.1)

The basic idea of kriging is to predict the value of a function at a given point by computing a weighted average of the known values of the function in the neighborhood of the point ([Wikipedia](https://en.wikipedia.org/wiki/Kriging)). 

The main application for scikit-gstat is variogram analysis and Kriging (Geostatistical analysis). The [Tutorial](https://scikit-gstat.readthedocs.io/en/latest/tutorials/01_getting_started.html) will guide you through the most basic functionality of scikit-gstat. There are other tutorials that will explain specific methods or attributes in scikit-gstat in more detail.

## 2. Import libraries <a class="anchor" id="2"></a>

[Back to Table of Contents](#0.1)

In [1]:
import numpy as np
import pandas as pd

# scikit-gstat = skgstat
import skgstat as skg
from skgstat import Variogram, OrdinaryKriging

# visualization
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## 3. Creation dataset <a class="anchor" id="3"></a>

[Back to Table of Contents](#0.1)

In [2]:
df = pd.read_pickle('../data/manipulated/geo_city.pkl')


In [3]:
mapping = {
    "Damaged": 3,
    "Destroyed": 4,
    "No visible damage": 1,
    "Possibly damaged": 2
}

df['damage_gra'] = df['damage_gra'].replace(mapping)

df.columns

Index(['obj_type', 'name', 'info', 'damage_gra', 'det_method', 'notation',
       'or_src_id', 'dmg_src_id', 'cd_value', 'real', 'geometry', 'province',
       'city', 'population', 'income', 'total_sales', 'second_sales',
       'water_access', 'elec_cons', 'building_perm', 'land_permited',
       'labour_fource', 'unemployment', 'agricultural', 'life_time',
       'hb_per100000', 'fertility', 'hh_size', 'point'],
      dtype='object')

In [ ]:
df

In [4]:
# Dataset visualization 
fig = plt.figure(figsize=(8, 8))
plt.scatter(df.latitude, df.longitude, 50, c=df.damage_gra, cmap='plasma')

AttributeError: 'GeoDataFrame' object has no attribute 'latitude'

<Figure size 800x800 with 0 Axes>

## 4. Variogram building <a class="anchor" id="4"></a>

[Back to Table of Contents](#0.1)

In [6]:
%%time
# Calculation variogram
V = skg.Variogram(coordinates=df[['latitude', 'longitude']].values, values=df['damage_gra'].values)
print(V)

MemoryError: Unable to allocate 387. GiB for an array with shape (51978456676,) and data type float64

In [ ]:
# Variogram visualization
V.plot()
plt.close

In [ ]:
# Visualization of the variogram with others parameters
V.n_lags = 10
V.maxlag = 50
V.bin_func = 'kmeans'
V.plot()
plt.close()

In [ ]:
# Visualization of the variograms for different models
fig, _a = plt.subplots(2,3, figsize=(18, 10), sharex=True, sharey=True)
axes = _a.flatten()
for i, model in enumerate(('spherical', 'exponential', 'gaussian', 'matern', 'stable', 'cubic')):
    V.model = model
    V.plot(axes=axes[i], hist=False, show=False)
    axes[i].set_title('Model: %s; RMSE: %.2f' % (model, V.rmse))

## 5. Visualization of the result of geostatistical data interpolation <a class="anchor" id="5"></a>

[Back to Table of Contents](#0.1)

In [ ]:
V.model = 'stable'
ok = OrdinaryKriging(V, min_points=3, max_points=5, mode='estimate')
xx, yy = np.mgrid[0:99:100j, 0:99:100j]
field = ok.transform(xx.flatten(), yy.flatten()).reshape(xx.shape)
s2 = ok.sigma.reshape(xx.shape)

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
art = axes[0].matshow(field.T, origin='lower', cmap='plasma')
axes[0].set_title('Interpolation')
axes[0].plot(df.x, df.y, '+k')
axes[0].set_xlim((0,100))
axes[0].set_ylim((0,100))
plt.colorbar(art, ax=axes[0])
art = axes[1].matshow(s2.T, origin='lower', cmap='YlGn_r')
axes[1].set_title('Kriging Error')
plt.colorbar(art, ax=axes[1])
axes[1].plot(df.x, df.y, '+w')
axes[1].set_xlim((0,100))
axes[1].set_ylim((0,100));

To increase the efficiency of the analysis, it is necessary to set the values not randomly, but on the basis of a certain mathematic function, adding noise so that the method can find existing patterns.

In [ ]:
def interpolate(V, ax):
    # Thanks to https://scikit-gstat.readthedocs.io/en/latest/tutorials/02_variogram_models.html
    
    xx, yy = np.mgrid[0:99:100j, 0:99:100j]
    ok = OrdinaryKriging(V, min_points=5, max_points=15, mode='exact')
    field = ok.transform(xx.flatten(), yy.flatten()).reshape(xx.shape)
    art = ax.matshow(field, origin='lower', cmap='plasma')
    ax.set_title('%s model' % V.model.__name__)
    plt.colorbar(art, ax=ax)
    return field

In [ ]:
fields = []
fig, _a = plt.subplots(2,3, figsize=(18, 12), sharex=True, sharey=True)
axes = _a.flatten()
for i, model in enumerate(('spherical', 'exponential', 'gaussian', 'matern', 'stable', 'cubic')):
    V.model = model
    fields.append(interpolate(V, axes[i]))

From the Kriging error map, you can see how the interpolation is very certain close to the observation points, but rather high in areas with only little coverage.

Models "matern" and "stable" work best with a very different data.

I hope you find this notebook useful and enjoyable.

Your comments and feedback are most welcome.

[Go to Top](#0)